# Problema de regresión

## Descripción del problema

En este problema vamos a trabajar con el conjunto de datos _Airfoil Self-Noise_, el cuál ha sido proporcionado por la NASA, y contiene los resultados de haber realizado un conjunto de pruebas aerodinámicas y acústicas en un túnel de viento sobre perfiles alares de dos y tres dimensiones.

El conjunto de datos está compuesto por 1503 filas y 6 columnas, los valores de las cuáles son todos números reales. Los datos de las 5 primeras columnas se corresponden con los datos de entrada, y la última columna se corresponde con la información de salida. A continuación se puede ver que representa cada uno de los atributos de forma ordenada:

1. Frecuencia, medida en $Hz$.
2. Ángulo de ataque (ángulo que forman la cuerda geométrica de un perfil alar con la dirección del aire incidente), medida en grados.
3. Longitud de la cuerda del perfil alar, medida en metros.
4. Velocidad _free-stream_, medida en metros por segundo. 
5. Distancia de desplazamiento de succión, medida en metros.
6. Nivel de presión sonora, medida en $dB$.


## Análisis de los datos

Antes de comenzar con todo el proceso de elección y selección de un modelo lineal, vamos a pararnos un momento para analizar los datos de los que disponemos con el fin de obtener más información sobre el problema.

Lo primero que tenemos que hacer es cargar los datos. Para ello, vamos a usar una función genérica que nos permita leer ficheros de datos y obtener un _DataFrame_ que podamos usar luego. Vamos a ver como sería esta función:

In [1]:
import numpy as np
import pandas as pd

def read_data_values(in_file, separator=None):
    """
    Funcion para leer los datos de un archivo
    
    :param in_file Archivo de entrada
    :param separator Separador que se utiliza en el archivo (por defecto
                     None)
    
    :return Devuelve los datos leidos del archivo en un DataFrame
    """
    
    # Cargar los datos en un DataFrame
    # Se indica que la primera columna no es el header
    df = pd.read_csv(in_file, sep=separator, header=None)
    
    return df

Con la función ya mostrada, vamos a cargar los datos:

In [2]:
df = read_data_values('datos/airfoil_self_noise.dat', separator='\t')

# Asignamos nombres a las columnas (según los atributos)
column_names = ['Frequency', 'Angle of attack', 'Chord length',
                'Free-stream velocity', 'SSD thickness', 'Sound Pressure']
df.columns = column_names

Con los datos ya cargados, vamos a obtener cierta información sobre éstos. En problemas de este tipo nos interesa
conocer por ejemplo si en ciertos casos faltan datos (no se ha podido obtener información sobre todos los
atributos debido a que es imposible hacerlo, han habido errores a la hora de tomarlos o no se disponía de las
herramientas necesarias), el número de valores distintos, los rangos de los datos (valores mínimos y máximos para
cada atributo), si existe algún tipo de correlación entre las variables, etc.

Vamos a comenzar estudiando primero las características más simples, para adentrarnos luego en el estudio de la correlación.

In [3]:
# Informacion sobre numero de filas, columnas y valores nulos
print('Num. de valores del conjunto de datos y valores faltantes:')
print(df.info())

# Obtener número de valores únicos por atributo
print('\nNumero de valores distintos por atributo')

for atribute in column_names:
    print(atribute + ': ' + str(df[atribute].unique().shape[0]))

# Mostrar los valores mínimos de cada atributo
print('\nValores minimos de cada atributo:')
print(df.min())

# Mostrar los valores máximos de cada atributo
print('\nValores maximos de cada atributo:')
print(df.max())

Num. de valores del conjunto de datos y valores faltantes:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1503 entries, 0 to 1502
Data columns (total 6 columns):
Frequency               1503 non-null int64
Angle of attack         1503 non-null float64
Chord length            1503 non-null float64
Free-stream velocity    1503 non-null float64
SSD thickness           1503 non-null float64
Sound Pressure          1503 non-null float64
dtypes: float64(5), int64(1)
memory usage: 70.5 KB
None

Numero de valores distintos por atributo
Frequency: 21
Angle of attack: 27
Chord length: 6
Free-stream velocity: 4
SSD thickness: 105
Sound Pressure: 1456

Valores minimos de cada atributo:
Frequency               200.000000
Angle of attack           0.000000
Chord length              0.025400
Free-stream velocity     31.700000
SSD thickness             0.000401
Sound Pressure          103.380000
dtype: float64

Valores maximos de cada atributo:
Frequency               20000.000000
Angle of attack  

Como se puede ver, tal y como se ha dicho al principio, el conjunto de datos está formado por 1503 muestras, cada
una de las cuáles tiene 6 valores (5 atributos de entrada y 1 de salida). Además, no falta información en ninguna
de las muestras, lo cuál es una buena noticia, ya que nos ahorra pensar como rellenar esa información restante.

Observando ahora los valores de los atributos, nos encontramos que, para los atributos de entrada, a pesar de
tener un número considerable de muestras, el número de valores únicos es casi insignificante; dicho de otra forma,
no hay mucha variabilidad en esta parte, ya que se repiten muchos de los valores. Para los valores de salida la
cosa cambia, ya que ahí sí que existe más variabilidad. Analizando ahora los rangos de los valores, vemos que hay atributos cuya variabilidad es mucho mayor que la del resto (como por ejemplo la frecuencia o la velocidad _free-stream_), mientras que hay algunos atributos cuya variabilidad es mucho menor. Esto nos indica que, o bien hay 
atributos que no varían mucho y se podrían eliminar, o bien hay _outliers_ en las muestras, lo cuál hace que
haya valores anómalos. Si miramos la salida, vemos que, a pesar de tener un montón de posibles valores, el rango de variabilidad no es tan grande como en otros atributos.